In [6]:
import csv
import pandas
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import chess
import chess.pgn

In [2]:
#Following Board Representation from Paper
def board_encoder(board):
    bit_boards = {"P": np.zeros(64),"N": np.zeros(64),"B": np.zeros(64),
                  "R": np.zeros(64),"Q": np.zeros(64),"K": np.zeros(64),
                  "p": np.zeros(64),"n": np.zeros(64),"b": np.zeros(64),
                  "r": np.zeros(64),"q": np.zeros(64),"k": np.zeros(64)}
    for i in range(0, 64):
        square = board.piece_at(i)
        if square is not None:
            bit_boards[square.symbol()][i] = 1
            
    bit_board = np.concatenate([bit_boards["P"], bit_boards["R"], bit_boards["N"], bit_boards["B"], 
                                bit_boards["Q"], bit_boards["K"], bit_boards["p"], bit_boards["r"],
                                bit_boards["n"], bit_boards["b"], bit_boards["q"], bit_boards["k"]])
    return bit_board


In [43]:
stockfish = pandas.read_csv('C:\\Users\\Christopher.Atkeson\\Documents\\web_project\\chess_evaluation\\stockfish.csv')
pgn = open('C:\\Users\\Christopher.Atkeson\\Documents\\web_project\\chess_evaluation\\data.pgn')
#stockfish = pandas.read_csv('/Users/chrisatkeson/Documents/web_project/chess_evaluation/stockfish.csv')
#pgn = open('/Users/chrisatkeson/Documents/web_project/chess_evaluation/data.pgn')

In [44]:
evaluations = stockfish['MoveScores'];
all_evaluations = [];
all_positions =[];

count = 0

for game in evaluations:
    count = count+1
    if count % 500 == 0:
        print(count)
    position = chess.pgn.read_game(pgn)
    for evaluation in game.split():
        all_positions.append(position.board())
        next_position = position.variations[0]
        position = next_position
        if evaluation.isdigit():
            all_evaluations.append([float(evaluation)])
        elif evaluation.lstrip('-').isdigit():
            all_evaluations.append([float(evaluation.lstrip('-'))*-1.0])
        else: 
            all_evaluations.append([0])

    if count == 5000:
        break
        
#normalize evaluations      
mean_evaluation = np.mean(all_evaluations)
stdeviation = np.std(all_evaluations)
max_evaluation = np.max(all_evaluations)
min_evaluation = np.min(all_evaluations)
max_abs_evaluation = max(max_evaluation, min_evaluation*-1.0)

normalized_evaluations = np.asarray(all_evaluations) * 1/max_abs_evaluation

#Constuct Board Vectors
positions = []
for position in all_positions:
    positions.append(board_encoder(position))

500
1000
1500
2000
2500
3000
3500
4000
4500
5000


In [36]:
#Following Board Representation from Paper
def board_encoder(board):
    bit_boards = {"P": np.zeros(64),"N": np.zeros(64),"B": np.zeros(64),
                  "R": np.zeros(64),"Q": np.zeros(64),"K": np.zeros(64),
                  "p": np.zeros(64),"n": np.zeros(64),"b": np.zeros(64),
                  "r": np.zeros(64),"q": np.zeros(64),"k": np.zeros(64)}
    for i in range(0, 64):
        square = board.piece_at(i)
        if square is not None:
            bit_boards[square.symbol()][i] = 1
            
    bit_board = np.concatenate([bit_boards["P"], bit_boards["R"], bit_boards["N"], bit_boards["B"], 
                                bit_boards["Q"], bit_boards["K"], bit_boards["p"], bit_boards["r"],
                                bit_boards["n"], bit_boards["b"], bit_boards["q"], bit_boards["k"]])
    return bit_board


In [53]:
#print(positions[0], normalized_evaluations[0])

In [ ]:
# Neural Net
learning_rate = 0.001
training_epochs = 5
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 500 # 1st layer number of neurons
n_hidden_2 = 500 # 2nd layer number of neurons
n_input = 768 # 12 8x8 chessboards
n_classes = 1 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, 1])

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Create model
def multilayer_perceptron(x):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Construct model
output = multilayer_perceptron(X)

# using mean squared error cost function
cost  = tf.reduce_mean(tf.square(output - Y))

# using Gradient Descent algorithm
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

In [52]:
with tf.Session() as sess:
    sess.run(init)
    cost_history=[]
    count = 0
    for step in np.arange(1):
        sess.run(optimizer, feed_dict={X:positions[0:100], Y:normalized_evaluations[0:100]})
        cost_history = np.append(cost_history, sess.run(cost,feed_dict={X:positions[0:100], Y:normalized_evaluations[0:100]}))
    
    pred_y = sess.run(output, feed_dict={X:positions})
    print (sess.run(output, feed_dict={X:positions}))



[[2.6995978e+08]
 [2.7529693e+08]
 [2.7613366e+08]
 ...
 [8.5603710e+06]
 [9.3569250e+06]
 [9.2662530e+06]]


In [37]:
print(normalized_evaluations)

[0.00145725 0.0013763  0.0009715  ... 0.83273964 0.83273964 0.83330635]
